In [2]:
import logging
import os
import random
from typing import Dict, List
import numpy as np
import torch
from torch.utils.data import random_split
from torch_geometric.loader import DataLoader as PyGDataLoader

from ase import Atoms
from mace.calculators import MACECalculator

# Import all necessary components from your other files
# Assumes these files have been updated as per our previous discussion
from models import DualReadoutMACE, compute_E_statistics_vectorized
from train import evaluate, DeltaEnergyLoss, load_data, pyg_collate, AtomsDataset
from ase.io import read
from Calculator import DeltaMaceCalculator

def get_vacuum_energies(calc_mace_off: MACECalculator, calc_mace_mp: MACECalculator, z_list: List[int]) -> Dict[int, float]:
    """Calculates the energy (mace_off) for single, isolated atoms."""
    logging.info("Calculating vacuum energies for regression baseline...")
    vacuum_energies = {}
    unique_atomic_numbers = sorted(list(set(z_list)))
    
    for z in unique_atomic_numbers:
        atom = Atoms(numbers=[z])
        atom.calc = calc_mace_off
        vacuum_ref = atom.get_potential_energy()
        atom.calc = calc_mace_mp
        vacuum_base = atom.get_potential_energy()
        vacuum_energies[z] = vacuum_ref - vacuum_base
        logging.info(f"  - Referance vacuum energy for Z={z}: {vacuum_ref:.4f} eV")
        logging.info(f"  - Base vacuum energy for Z={z}: {vacuum_base:.4f} eV")

        
    return vacuum_energies




base_model_path = "MACE-MP_small.model"
high_accuracy_model_path = "MACE-OFF24_medium.model"
    
device_str = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_str)

atoms = read('/home/krystynasyty/X23/08_cyanamide/solid.xyz')
full_atoms_list = []
full_atoms_list.append(atoms)

base_mace_model = torch.load(base_model_path, map_location=device, weights_only=False)
base_mace_model.to(dtype=torch.float64).eval()

r_max = base_mace_model.r_max.item()
atomic_numbers_list = base_mace_model.atomic_numbers.tolist()
z_map = {z: i for i, z in enumerate(atomic_numbers_list)}
n_species = len(atomic_numbers_list)
    
calc_mace_off = MACECalculator(model_path=high_accuracy_model_path, device=device_str)
calc_mace_mp = MACECalculator(model_path=base_model_path, device=device_str)
all_zs_in_dataset = [z for atoms in full_atoms_list for z in atoms.get_atomic_numbers()]
vacuum_energies = get_vacuum_energies(calc_mace_off, calc_mace_mp, all_zs_in_dataset)

atoms.calc = calc_mace_off


E_total = np.array(atoms.get_potential_energy())
N_atoms = np.array([len(atoms) for atoms in full_atoms_list])
    
X_table = np.zeros((len(full_atoms_list), n_species), dtype=np.int64)
for i, atoms in enumerate(full_atoms_list):
    for z in atoms.get_atomic_numbers():
        idx = z_map.get(z)
        if idx is not None:
            X_table[i, idx] += 1
    
epera_regression_shifts = compute_E_statistics_vectorized(
    E=E_total, N=N_atoms, X=X_table, n_species=n_species, 
    delta_vacuum_energies=vacuum_energies, z_map=z_map
)
    
shifts_for_model = torch.tensor(epera_regression_shifts, dtype=torch.float64, device=device)

model = DualReadoutMACE(base_mace_model, vacuum_energy_shifts=shifts_for_model)
model.to(device)


calculator = DeltaMaceCalculator(model=model, device="cpu")
atoms = read('/home/krystynasyty/X23/08_cyanamide/solid.xyz')
print("Mace-off calc:")
atoms.calc = calc_mace_off
E_off = atoms.get_potential_energy()
print("Energy: ", atoms.get_potential_energy())
print("Forces: ", atoms.get_forces())
print("Mace dual calc:")
atoms.calc = calculator
E_calc = atoms.get_potential_energy()
print("Energy: ", atoms.get_potential_energy())
print("Forces: ", atoms.get_forces())
print("Energy diferance: ", E_off-E_calc)

2025-10-17 14:21:39,720 - WARNING - 'model_path' argument is deprecated, please use 'model_paths' in the future.
/home/krystynasyty/myenv/lib/python3.12/site-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)
2025-10-17 14:21:39,773 - INFO - Using CPU
2025-10-17 14:21:39,774 - WARNING - 'model_path' argument is deprecated, please use 'model_paths' in the future.
/home/krystynasyty/myenv/lib/python3.12/site-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)
2025-10-17 14:21:39,820 - INFO - Using CPU
2025-10-17 14:21:39,821 - INFO - Calculating vacuum energies f

Using head Default out of ['Default']
No dtype selected, switching to float64 to match model dtype.
Using head Default out of ['Default']
No dtype selected, switching to float64 to match model dtype.


2025-10-17 14:21:39,975 - INFO -   - Base vacuum energy for Z=1: -1.2136 eV
2025-10-17 14:21:40,102 - INFO -   - Referance vacuum energy for Z=6: -1030.5672 eV
2025-10-17 14:21:40,103 - INFO -   - Base vacuum energy for Z=6: -1.9194 eV
2025-10-17 14:21:40,181 - INFO -   - Referance vacuum energy for Z=7: -1486.3750 eV
2025-10-17 14:21:40,182 - INFO -   - Base vacuum energy for Z=7: -2.8496 eV
2025-10-17 14:21:40,641 - INFO - Performing vectorized regression to find optimal energy shifts...
2025-10-17 14:21:40,642 - INFO - Regression complete. Final shifts calculated.
2025-10-17 14:21:40,645 - INFO - Using CPU


[  -19.46974196     0.             0.             0.
     0.         -1032.2034013  -1490.63677872     0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.         

In [3]:
from ase import Atoms
import numpy as np

print("____Testy niezmienniczości energii ze względu na permutajcę atomów_____")

atoms = Atoms('H2O', positions=[(0.1, 0.2, 0.3), (0.9, -0.1, 0.2), (-0.5, 0.4, -0.6)])

base_model_path = "MACE-MP_small.model"
high_accuracy_model_path = "MACE-OFF24_medium.model"
    
device_str = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_str)

full_atoms_list = []
full_atoms_list.append(atoms)

base_mace_model = torch.load(base_model_path, map_location=device, weights_only=False)
base_mace_model.to(dtype=torch.float64).eval()

r_max = base_mace_model.r_max.item()
atomic_numbers_list = base_mace_model.atomic_numbers.tolist()
z_map = {z: i for i, z in enumerate(atomic_numbers_list)}
n_species = len(atomic_numbers_list)
    
calc_mace_off = MACECalculator(model_path=high_accuracy_model_path, device=device_str)
calc_mace_mp = MACECalculator(model_path=base_model_path, device=device_str)
all_zs_in_dataset = [z for atoms in full_atoms_list for z in atoms.get_atomic_numbers()]
vacuum_energies = get_vacuum_energies(calc_mace_off, calc_mace_mp, all_zs_in_dataset)

atoms.calc = calc_mace_off


E_total = np.array(atoms.get_potential_energy())
N_atoms = np.array([len(atoms) for atoms in full_atoms_list])
    
X_table = np.zeros((len(full_atoms_list), n_species), dtype=np.int64)
for i, atoms in enumerate(full_atoms_list):
    for z in atoms.get_atomic_numbers():
        idx = z_map.get(z)
        if idx is not None:
            X_table[i, idx] += 1
    
epera_regression_shifts = compute_E_statistics_vectorized(
    E=E_total, N=N_atoms, X=X_table, n_species=n_species, 
    delta_vacuum_energies=vacuum_energies, z_map=z_map
)
    
shifts_for_model = torch.tensor(epera_regression_shifts, dtype=torch.float64, device=device)

model = DualReadoutMACE(base_mace_model, vacuum_energy_shifts=shifts_for_model)
model.to(device)


calculator = DeltaMaceCalculator(model=model, device="cpu")



atoms.calc = calculator

try:
    original_energy = atoms.get_potential_energy()
    print(f"Energia początkowa: {original_energy:.6f} eV")
except Exception as e:
    print(f"Błąd podczas obliczania energii początkowej: {e}")

        
print("-" * 30)

print("Test niezmienniczości permutacyjnej...")
permuted_molecule = atoms.copy()

indices = list(range(len(permuted_molecule)))
indices[0], indices[1] = indices[1], indices[0]
permuted_molecule = permuted_molecule[indices]

permuted_molecule.calc = calculator
permuted_energy = permuted_molecule.get_potential_energy()

print(f"Energia po permutacji atomów H: {permuted_energy:.6f} eV")

# Sprawdzenie, czy energie są takie same (z małą tolerancją numeryczną)
if np.allclose(original_energy, permuted_energy):
    print("✅ Test permutacji ZDANY")
else:
    print("❌ Test permutacji NIEZDANY")
        
print("-" * 30)

print("Test niezmienniczości rotacyjnej...")
rotated_molecule = atoms.copy()

    # Obróć cząsteczkę o dowolne kąty, np. 33 stopnie wokół osi 'x' i 58 wokół 'y'
rotated_molecule.rotate(33, 'x', center='COM')
rotated_molecule.rotate(58, 'y', center='COM')

    # Upewnij się, że kalkulator jest przypisany do obróconej cząsteczki
rotated_molecule.calc = calculator
rotated_energy = rotated_molecule.get_potential_energy()

print(f"Energia po rotacji: {rotated_energy:.6f} eV")

    # Sprawdzenie, czy energie są takie same
if np.allclose(original_energy, rotated_energy):
    print("✅ Test rotacji ZDANY")
else:
    print("❌ Test rotacji NIEZDANY")
        
print("-" * 30)


2025-10-17 14:21:57,254 - WARNING - 'model_path' argument is deprecated, please use 'model_paths' in the future.
/home/krystynasyty/myenv/lib/python3.12/site-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)
2025-10-17 14:21:57,307 - INFO - Using CPU
2025-10-17 14:21:57,308 - WARNING - 'model_path' argument is deprecated, please use 'model_paths' in the future.
/home/krystynasyty/myenv/lib/python3.12/site-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)
2025-10-17 14:21:57,353 - INFO - Using CPU
2025-10-17 14:21:57,353 - INFO - Calculating vacuum energies f

____Testy niezmienniczości energii ze względu na permutajcę atomów_____
Using head Default out of ['Default']
No dtype selected, switching to float64 to match model dtype.
Using head Default out of ['Default']
No dtype selected, switching to float64 to match model dtype.


2025-10-17 14:21:57,502 - INFO -   - Referance vacuum energy for Z=1: -13.5720 eV
2025-10-17 14:21:57,502 - INFO -   - Base vacuum energy for Z=1: -1.2136 eV
2025-10-17 14:21:57,715 - INFO -   - Referance vacuum energy for Z=8: -2043.9337 eV
2025-10-17 14:21:57,716 - INFO -   - Base vacuum energy for Z=8: -1.8437 eV
2025-10-17 14:21:57,785 - INFO - Performing vectorized regression to find optimal energy shifts...
2025-10-17 14:21:57,786 - INFO - Regression complete. Final shifts calculated.
2025-10-17 14:21:57,788 - INFO - Using CPU


[  -16.09310664     0.             0.             0.
     0.             0.             0.         -2043.95732993
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0. 

In [5]:
atoms = Atoms('HH', positions=[(0, 0, 0), (100, 0, 0)])

atoms.calc = calc_mace_off
print(atoms.get_potential_energy())
atoms.calc = calculator
print(atoms.get_potential_energy())
atoms=Atoms('H')
atoms.calc = calculator
print(2*atoms.get_potential_energy())


-27.143929545293837
-34.613329046407145
-34.613329046407145
